# 4.1 Vim文本编辑器
>命令模式：控制光标移动，可对文本进行复制、粘贴、删除和查找等工作。
>输入模式：正常的文本录入。
>末行模式：保存或退出文档，以及设置编辑环境。
![](https://www.linuxprobe.com/wp-content/uploads/2015/03/vim%E4%B8%8D%E5%90%8C%E6%A8%A1%E5%BC%8F%E9%97%B4%E7%9A%84%E5%88%87%E6%8D%A2.png)

Vim中常用的命令

命令|作用
----|----
dd|删除（剪切）光标所在整行
5dd|删除（剪切）从光标处开始的5行
yy|复制光标所在整行
5yy|复制从光标处开始的5行
n|显示搜索命令定位到的下一个字符串
N|显示搜索命令定位到的上一个字符串
u|撤销上一步的操作
p|将之前删除（dd）或复制（yy）过的数据粘贴到光标后面

末行模式中可用的命令

命令|作用
----|----
:w|保存
:q|退出
:q!|强制退出（放弃对文档的修改内容）
:wq!|强制保存退出
:set nu|显示行号
:set nonu|不显示行号
:命令|执行该命令
:整数|跳转到该行
:s/one/two|将当前光标所在行的第一个one替换成two
:s/one/two/g|将当前光标所在行的所有one替换成two
:%s/one/two/g|将全文中的所有one替换成two
?字符串|在文本中从下至上搜索该字符串
/字符串|在文本中从上至下搜索该字符串

## 4.1.1 编写简单文档
## 4.1.2 配置主机名称
为了便于在局域网中查找某台特定的主机，或者对主机进行区分，除了要有IP地址外，还要为主机配置一个主机名，主机之间可以通过这个类似于域名的名称来相互访问。在Linux系统中，主机名大多保存在/etc/hostname文件中，接下来将/etc/hostname文件的内容修改为“linuxprobe.com”，步骤如下。
- 第1步：使用Vim编辑器修改“/etc/hostname”主机名称文件。
- 第2步：把原始主机名称删除后追加“linuxprobe.com”。注意，使用Vim编辑器修改主机名称文件后，要在末行模式下执行:wq!命令才能保存并退出文档。
- 第3步：保存并退出文档，然后使用hostname命令检查是否修改成功。

## 4.1.3 配置网卡信息
网卡IP地址配置的是否正确是两台服务器是否可以相互通信的前提。在Linux系统中，一切都是文件，因此配置网络服务的工作其实就是在编辑网卡配置文件

现在有一个名称为ifcfg-eno16777736的网卡设备，我们将其配置为开机自启动，并且IP地址、子网、网关等信息由人工指定，其步骤应该如下所示。
- 第1步：首先切换到/etc/sysconfig/network-scripts目录中（存放着网卡的配置文件）。
- 第2步：使用Vim编辑器修改网卡文件ifcfg-eno16777736，逐项写入下面的配置参数并保存退出。由于每台设备的硬件及架构是不一样的，因此请读者使用ifconfig命令自行确认各自网卡的默认名称。
    - 设备类型：TYPE=Ethernet
    - 地址分配模式：BOOTPROTO=static
    - 网卡名称：NAME=eno16777736
    - 是否启动：ONBOOT=yes
    - IP地址：IPADDR=192.168.10.10
    - 子网掩码：NETMASK=255.255.255.0
    - 网关地址：GATEWAY=192.168.10.1
    - DNS地址：DNS1=192.168.10.1
- 第3步：重启网络服务并测试网络是否联通。
进入到网卡配置文件所在的目录，然后编辑网卡配置文件，在其中填入下面的信息：
```bash
[root@linuxprobe ~]# cd /etc/sysconfig/network-scripts/
[root@linuxprobe network-scripts]# vim ifcfg-eno16777736
TYPE=Ethernet
BOOTPROTO=static
NAME=eno16777736
ONBOOT=yes
IPADDR=192.168.10.10
NETMASK=255.255.255.0
GATEWAY=192.168.10.1
DNS1=192.168.10.1
```
执行重启网卡设备的命令（在正常情况下不会有提示信息），然后通过ping命令测试网络能否联通。由于在Linux系统中ping命令不会自动终止，因此需要手动按下Ctrl-c键来强行结束进程。
```bash
[root@linuxprobe network-scripts]# systemctl restart network
[root@linuxprobe network-scripts]# ping 192.168.10.10
PING 192.168.10.10 (192.168.10.10) 56(84) bytes of data.
64 bytes from 192.168.10.10: icmp_seq=1 ttl=64 time=0.081 ms
64 bytes from 192.168.10.10: icmp_seq=2 ttl=64 time=0.083 ms
64 bytes from 192.168.10.10: icmp_seq=3 ttl=64 time=0.059 ms
64 bytes from 192.168.10.10: icmp_seq=4 ttl=64 time=0.097 ms
^C
--- 192.168.10.10 ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 2999ms
rtt min/avg/max/mdev = 0.059/0.080/0.097/0.013 ms
```

## 4.1.4 配置Yum仓库
搭建并配置Yum软件仓库的大致步骤如下所示。
- 第1步：进入到/etc/yum.repos.d/目录中（因为该目录存放着Yum软件仓库的配置文件）。
- 第2步：使用Vim编辑器创建一个名为rhel7.repo的新配置文件（文件名称可随意，但后缀必须为.repo），逐项写入下面加粗的配置参数并保存退出（不要写后面的中文注释）。
    - [rhel-media] ：Yum软件仓库唯一标识符，避免与其他仓库冲突。
    - name=linuxprobe：Yum软件仓库的名称描述，易于识别仓库用处。
    - baseurl=file:///media/cdrom： 提供的方式包括FTP（ftp://..）、HTTP（http://..）、本地（file:///..）。
    - enabled=1：设置此源是否可用；1为可用，0为禁用。
    - gpgcheck=1：设置此源是否校验文件；1为校验，0为不校验。
    - gpgkey=file:///media/cdrom/RPM-GPG-KEY-redhat-release： 若上面参数开启校验，那么请指定公钥文件地址。
- 第3步：按配置参数的路径挂载光盘，并把光盘挂载信息写入到/etc/fstab文件中。
- 第4步：使用“yum install httpd -y”命令检查Yum软件仓库是否已经可用。
进入/etc/yum.repos.d目录中后创建Yum配置文件：
```bash
[root@linuxprobe ~]# cd /etc/yum.repos.d/
[root@linuxprobe yum.repos.d]# vim rhel7.repo
[rhel7]
name=rhel7
baseurl=file:///media/cdrom
enabled=1
gpgcheck=0
```

# 4.2 编写Shell脚本
Shell脚本命令的工作方式有两种：交互式和批处理。
- 交互式（Interactive）：用户每输入一条命令就立即执行。
- 批处理（Batch）：由用户事先编写好一个完整的Shell脚本，Shell会一次性执行脚本中诸多的命令。

## 4.2.1 编写简单的脚本
使用Vim编辑器把Linux命令按照顺序依次写入到一个文件中，这就是一个简单的脚本了。

例如，如果想查看当前所在工作路径并列出当前目录下所有的文件及属性信息，实现这个功能的脚本应该类似于下面这样：
```bash
[root@linuxprobe ~]# vim example.sh
#!/bin/bash 
#For Example BY linuxprobe.com 
pwd 
ls -al
```
Shell脚本文件的名称可以任意，但为了避免被误以为是普通文件，建议**将.sh后缀加上，以表示是一个脚本文件。在上面的这个example.sh脚本中实际上出现了三种不同的元素：第一行的脚本声明（#!）用来告诉系统使用哪种Shell解释器来执行该脚本；第二行的注释信息（#）是对脚本功能和某些命令的介绍信息，使得自己或他人在日后看到这个脚本内容时，可以快速知道该脚本的作用或一些警告信息；第三、四行的可执行语句也就是我们平时执行的Linux命令了。**





## 4.2.2 接收用户的参数
**\$0对应的是当前Shell脚本程序的名称，$#对应的是总共有几个参数，$*对应的是所有位置的参数值，\$?对应的是显示上一次命令的执行返回值，而\$1、\$2、\$3……则分别对应着第N个位置的参数值**
![](https://www.linuxprobe.com/wp-content/uploads/2015/07/Shell%E8%84%9A%E6%9C%AC%E7%A8%8B%E5%BA%8F%E4%B8%AD%E7%9A%84%E5%8F%82%E6%95%B0%E4%BD%8D%E7%BD%AE%E5%8F%98%E9%87%8F-768x140.png)
理论过后我们来练习一下。尝试编写一个脚本程序示例，通过引用上面的变量参数来看下真实效果：
```bash
[root@linuxprobe ~]# vim example.sh
#!/bin/bash
echo "当前脚本名称为$0"
echo "总共有$#个参数，分别是$*。"
echo "第1个参数为$1，第5个为$5。"
[root@linuxprobe ~]# sh example.sh one two three four five six
当前脚本名称为example.sh
总共有6个参数，分别是one two three four five six。
第1个参数为one，第5个为five。
```

## 4.2.3 判断用户的参数
统在执行mkdir命令时会判断用户输入的信息，即判断用户指定的文件夹名称是否已经存在，如果存在则提示报错；反之则自动创建。Shell脚本中的条件测试语法可以判断表达式是否成立，若条件成立则返回数字0，否则便返回其他随机数值。
![](https://www.linuxprobe.com/wp-content/uploads/2015/07/%E6%B5%8B%E8%AF%95%E8%AF%AD%E5%8F%A5%E6%A0%BC%E5%BC%8F.png)
条件测试语句可以分为4种：
>文件测试语句；
>逻辑测试语句；
>整数值比较语句；
>字符串比较语句。

文件测试所用的参数

操作符|作用
------|----
-d|测试文件是否为目录类型
-e|测试文件是否存在
-f|判断是否为一般文件
-r|测试当前用户是否有权限读取
-w|测试当前用户是否有权限写入
-x|测试当前用户是否有权限执行

使用文件测试语句来判断/etc/fstab是否为一个目录类型的文件，然后通过Shell解释器的内设$?变量显示上一条命令执行后的返回值。如果返回值为0，则目录存在；如果返回值为非零的值，则意味着目录不存在：

```bash
[root@linuxprobe ~]# [ -d /etc/fstab ]
[root@linuxprobe ~]# echo $?
1

再使用文件测试语句来判断/etc/fstab是否为一般文件，如果返回值为0，则代表文件存在，且为一般文件：

```bash
[root@linuxprobe ~]# [ -f /etc/fstab ]
[root@linuxprobe ~]# echo $?
0
```

逻辑语句用于对测试结果进行逻辑分析，根据测试结果可实现不同的效果。例如在Shell终端中逻辑“与”的运算符号是&&，它表示当前面的命令执行成功后才会执行它后面的命令，因此可以用来判断/dev/cdrom文件是否存在，若存在则输出Exist字样。
```bash
[root@linuxprobe ~]# [ -e /dev/cdrom ] && echo "Exist"
Exist
```
除了逻辑“与”外，还有逻辑“或”，它在Linux系统中的运算符号为||，表示当前面的命令执行失败后才会执行它后面的命令，因此可以用来结合系统环境变量USER来判断当前登录的用户是否为非管理员身份：
```bash
[root@linuxprobe ~]# echo $USER
root
[root@linuxprobe ~]# [ $USER = root ] || echo "user"
[root@linuxprobe ~]# su - linuxprobe 
[linuxprobe@linuxprobe ~]$ [ $USER = root ] || echo "user"
user
```
第三种逻辑语句是“非”，在Linux系统中的运算符号是一个叹号（！），它表示把条件测试中的判断结果取相反值。也就是说，如果原本测试的结果是正确的，则将其变成错误的；原本测试错误的结果则将其变成正确的。

我们现在切换回到root管理员身份，再判断当前用户是否为一个非管理员的用户。由于判断结果因为两次否定而变成正确，因此会正常地输出预设信息：
```bash
[linuxprobe@linuxprobe ~]$ exit
logout
[root@linuxprobe root]# [ $USER != root ] || echo "administrator"
administrator
```
当前我们正在登录的即为管理员用户—root。下面这个示例的执行顺序是，先判断当前登录用户的USER变量名称是否等于root，然后用逻辑运算符“非”进行取反操作，效果就变成了判断当前登录的用户是否为非管理员用户了。最后若条件成立则会根据逻辑“与”运算符输出user字样；或条件不满足则会通过逻辑“或”运算符输出root字样，而如果前面的&&不成立才会执行后面的||符号。
```bash
[root@linuxprobe ~]# [ $USER != root ] && echo "user" || echo "root"
root
```

整数比较运算符仅是对数字的操作，不能将数字与字符串、文件等内容一起操作，而且不能想当然地使用日常生活中的等号、大于号、小于号等来判断。因为等号与赋值命令符冲突，大于号和小于号分别与输出重定向命令符和输入重定向命令符冲突。因此一定要使用规范的整数比较运算符来进行操作。
                                                 可用的整数比较运算符

操作符|作用
------|----
-eq|是否等于
-ne|是否不等于
-gt|是否大于
-lt|是否小于
-le|是否等于或小于
-ge|是否大于或等于

测试一下10是否大于10以及10是否等于10（通过输出的返回值内容来判断）：
```bash
[root@linuxprobe ~]# [ 10 -gt 10 ]
[root@linuxprobe ~]# echo $?
1
[root@linuxprobe ~]# [ 10 -eq 10 ]
[root@linuxprobe ~]# echo $?
0
```

接下来先使用free -m命令查看内存使用量情况（单位为MB），然后通过grep Mem:命令过滤出剩余内存量的行，再用awk '{print \$4}'命令只保留第四列，最后用FreeMem=`语句`的方式把语句内执行的结果赋值给变量

```bash
[root@linuxprobe ~]# free -m
            total     used     free     shared     buffers     cached
Mem:        1826      1244     582      9          1           413
-/+ buffers/cache:    830 996
Swap:       2047      0        2047
[root@linuxprobe ~]# free -m | grep Mem:
Mem:        1826      1244     582      9 
[root@linuxprobe ~]# free -m | grep Mem: | awk '{print $4}'
582
[root@linuxprobe ~]# FreeMem=`free -m | grep Mem: | awk '{print $4}'`
[root@linuxprobe ~]# echo $FreeMem 
582
```
上面用于获取内存可用量的命令以及步骤可能有些“超纲”了，如果不能理解领会也不用担心，接下来才是重点。我们使用整数运算符来判断内存可用量的值是否小于1024，若小于则会提示“Insufficient Memory”（内存不足）的字样：
```bash
[root@linuxprobe ~]# [ $FreeMem -lt 1024 ] && echo "Insufficient Memory"
Insufficient Memory
```
字符串比较语句用于判断测试字符串是否为空值，或两个字符串是否相同。它经常用来判断某个变量是否未被定义（即内容为空值），理解起来也比较简单。

常见的字符串比较运算符

操作符|作用
------|----
=|比较字符串内容是否相同
!=|比较字符串内容是否不同
-z|判断字符串内容是否为空

接下来通过判断String变量是否为空值，进而判断是否定义了这个变量：
```bash
[root@linuxprobe ~]# [ -z $String ]
[root@linuxprobe ~]# echo $?
0
```
再尝试引入逻辑运算符来试一下。当用于保存当前语系的环境变量值LANG不是英语（en.US）时，则会满足逻辑测试条件并输出“Not en.US”（非英语）的字样：
```bash
[root@linuxprobe ~]# echo $LANG
en_US.UTF-8
[root@linuxprobe ~]# [ $LANG != "en.US" ] && echo "Not en.US"
Not en.US
```